# Data transformations with Konduit and DataVec

In [2]:
from konduit import *
from konduit.json_utils import config_to_dict_with_type
from konduit.server import Server
from konduit.client import Client
from konduit.utils import is_port_in_use

import random
import time
import json
import os 

# from jnius import autoclass

# # import necessary Java classes
# TransformProcessBuilder = autoclass(
#     'org.datavec.api.transform.TransformProcess$Builder'
# )

# SchemaBuilder = autoclass(
#     'org.datavec.api.transform.schema.Schema$Builder'
# )

# inference_configuration_java_class = autoclass(
#     'ai.konduit.serving.InferenceConfiguration'
# )

# TransformProcess = autoclass(
#     'org.datavec.api.transform.TransformProcess'
# )

# StringJava = autoclass("java.lang.String")

# Configuration 

## Server configuration 

### Data transformations 

#### Schema

A Schema specifies ... 

In [3]:
import pydatavec

from pydatavec import Schema, TransformProcess

schema = Schema()
schema.add_string_column("first")

tp = TransformProcess(schema)
tp.append_string("first", "two")

In [4]:
java_tp = tp.to_java()

pdl4j: required uberjar not found, building with docker...
Docker available. Starting build...


Exception: Subprocess error for command: ['docker', 'build', 'C:\\Users\\Skymind AI Berhad\\.deeplearning4j\\pydl4j', '-t', 'pydl4j']

In [ ]:
tp_json = java_tp.toJson()
load_java_tp(tp_json)
_ = json.dumps(tp_json)
as_python_json = json.loads(tp_json)

In [2]:
schema = (SchemaBuilder()
          .addColumnString(StringJava('first'))
          .build())

#### TransformProcess

In [3]:
tp = (TransformProcessBuilder(schema)
      .appendStringColumnTransform(
          StringJava("first"), 
          StringJava("two"))
      .build())

tp_json = tp.toJson()
tp_python_json = json.loads(tp_json)

### Pipeline step

In [28]:
from konduit import TransformProcessPipelineStep 

input_names = ['default']
output_names = ['default']

transform_process = TransformProcessPipelineStep(
    input_schemas = {"default": ["String"]}, 
    output_schemas = {"default": ["String"]}, 
    input_names = input_names, 
    output_names = output_names, 
    input_column_names = {"default": ["first"]}, 
    output_column_names = {"default": ["first"]}, 
    transform_processes = {"default": tp_dict}
)

### Inference configuration

In [29]:
import numpy as np 
from konduit import ServingConfig, InferenceConfiguration
from konduit.server import Server
port = np.random.randint(1000, 65535)
serving_config = ServingConfig(
    http_port=port,
    input_data_type='JSON',
    output_data_type='JSON',
)

inference_config = InferenceConfiguration(
    serving_config=serving_config,
    pipeline_steps=[transform_process]
)

server = Server(
    inference_config=inference_config
)

In [30]:
inference_config.as_dict()

{'@type': 'InferenceConfiguration',
 'pipelineSteps': [{'@type': 'TransformProcessPipelineStep',
   'inputSchemas': {'default': ['String']},
   'outputSchemas': {'default': ['String']},
   'inputNames': ['default'],
   'outputNames': ['default'],
   'inputColumnNames': {'default': ['first']},
   'outputColumnNames': {'default': ['first']},
   'transformProcesses': {'default': {'steps': [('appendStringColumnTransform',
       'first',
       'two')],
     'schema': {'column_names': ['first'], 'meta': [['string']]}}}}],
 'servingConfig': {'@type': 'ServingConfig',
  'httpPort': 7645,
  'inputDataType': 'JSON',
  'outputDataType': 'JSON',
  'logTimings': True}}

## Configuring the Client

In [31]:
from konduit.client import Client 
client = Client(input_names=input_names,
                output_names=output_names,
                return_output_type='JSON',
                input_type='JSON',
                endpoint_output_type='RAW',
                url='http://localhost:' + str(port))

# Running the server

In [35]:
import time 
server.start()
time.sleep(30)
data_input = {'first': 'value'}
predicted = client.predict(data_input)
print(predicted)
server.stop()

ConnectionError: HTTPConnectionPool(host='localhost', port=7645): Max retries exceeded with url: /raw/json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001B4B2B79A48>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [34]:
server.stop()

In [22]:
pydatavec.__version__

AttributeError: module 'pydatavec' has no attribute '__version__'